In [1]:
import pandas as pd
import folium

Download data acceslibre

In [2]:
url_access_libre = "https://www.data.gouv.fr/fr/datasets/r/5b0f44f2-e6ea-4a58-874d-6fe364b40342"
df_accesslibre = pd.read_csv(url_access_libre)

/tmp/ipykernel_50281/489921021.py:2: DtypeWarning: Columns (56,67,68,69,70,71,72,73,74,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accesslibre = pd.read_csv(url_access_libre)


In [3]:
df_accesslibre.columns

Index(['id', 'name', 'postal_code', 'commune', 'numero', 'voie', 'lieu_dit',
       'code_insee', 'siret', 'activite', 'contact_url', 'site_internet',
       'longitude', 'latitude', 'transport_station_presence',
       'stationnement_presence', 'stationnement_pmr',
       'stationnement_ext_presence', 'stationnement_ext_pmr',
       'cheminement_ext_presence', 'cheminement_ext_terrain_stable',
       'cheminement_ext_plain_pied', 'cheminement_ext_ascenseur',
       'cheminement_ext_nombre_marches', 'cheminement_ext_reperage_marches',
       'cheminement_ext_sens_marches', 'cheminement_ext_main_courante',
       'cheminement_ext_rampe', 'cheminement_ext_pente_presence',
       'cheminement_ext_pente_degre_difficulte',
       'cheminement_ext_pente_longueur', 'cheminement_ext_devers',
       'cheminement_ext_bande_guidage', 'cheminement_ext_retrecissement',
       'entree_reperage', 'entree_vitree', 'entree_vitree_vitrophanie',
       'entree_plain_pied', 'entree_ascenseur', 'entree_mar

Columns selection

In [4]:
df_balises = df_accesslibre[['id', 'name', 'code_insee', 'longitude', 'latitude', 'entree_balise_sonore']]


Suppression des établissements sans information sur les balises sonores

In [5]:
df_balises = df_balises[df_balises['entree_balise_sonore'].notna()]
print(df_balises.shape)

(65905, 6)


In [6]:
df_balises['entree_balise_sonore'].value_counts()

entree_balise_sonore
False    63578
True      2327
Name: count, dtype: int64

In [7]:
df_balises

,id,name,code_insee,longitude,latitude,entree_balise_sonore
4,c7d781c0-12cf-4d66-aeef-571d8d4efd4f,0'mok@bar,75118,2.359299,48.893598,False
5,893019fe-cf9e-405e-9423-91955766cb45,0RPI,76322,1.050465,49.408801,False
6,983f2d7a-3056-4a2a-b5b5-2b8ff0b40234,0zo,NaN,2.248571,48.896493,False
9,e58f3350-8ae9-4f95-8343-98b2d22cd070,1000 BORNES,14220,0.075918,49.357996,False
11,128b173b-71c1-4e86-bf70-91122cee51ec,1000 Club,88109,6.676431,48.051255,False
...,...,...,...,...,...,...
526064,c6cd1343-3c94-4b95-84c5-be67b1a15fda,ZXSBN6DD3VZBGSY,NaN,2.349630,48.869574,False
526069,2ae3f50d-3ecb-4a82-b677-60f0661b58f8,zyke piscine,NaN,-0.626179,44.807678,False
526070,0c76f896-72a0-484c-96de-92d9349be617,Zyke Piscine,NaN,0.180574,45.692716,False
526071,e61427a8-da8e-474f-9fa7-e9403ac75de5,Zyke-Piscine Internationale,NaN,0.694818,44.170681,False


Cleaning des positions GPS

In [8]:
# Convertir les colonnes latitude et longitude si nécessaire
df_balises['longitude'] = pd.to_numeric(df_balises['longitude'], errors='coerce')
df_balises['latitude'] = pd.to_numeric(df_balises['latitude'], errors='coerce')

# Supprimer les lignes avec des coordonnées invalides
df_balises = df_balises.dropna(subset=['longitude', 'latitude'])


On ne garde que les établissements d'une communes, et uniquement ceux avec une borne sonore. 

In [9]:
df_balises_true = df_balises[df_balises["entree_balise_sonore"]]
df_balises_true = df_balises[df_balises["code_insee"]=="85294"]


In [10]:
point_central = [df_balises_true['latitude'].mean(), df_balises_true['longitude'].mean()]
carte = folium.Map(location=point_central, zoom_start=6)

for _, row in df_balises_true.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['name']}</b>",
    ).add_to(carte)

display(carte)


In [16]:
df_accesslibre[df_accesslibre["name"].str.contains("insee", case=False)]

,id,name,postal_code,commune,numero,voie,lieu_dit,code_insee,siret,activite,...,accueil_chambre_numero_visible,accueil_chambre_equipement_alerte,accueil_chambre_accompagnement,accueil_retrecissement,sanitaires_presence,sanitaires_adaptes,labels,labels_familles_handicap,registre_url,conformite
157680,65d1cada-aaef-4597-9036-7d40cc0cd22d,Direction régionale de l'Insee - Hauts-de-Fran...,59800,Lille,130,Avenue du Président John Fitzgerald Kennedy,NaN,59350,NaN,Administration publique,...,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN
230482,e7de612e-4fec-42bb-be02-d40d6f0ac16f,INSEEC BTS Paris,75003,Paris,197,Rue du Temple,NaN,75103,NaN,Collège,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df_accesslibre[df_accesslibre["voie"].str.contains("verdier", case=False)]

ValueError: Cannot mask with non-boolean array containing NA / NaN values